In [1]:
# import libraries

from utils import text_processing
from utils import get_weather
import datetime

import requests

import pandas as pd

from bs4 import BeautifulSoup

import re

import nltk
import os
import string
import numpy as np
import copy 
import pickle
import re
import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



import pandas as pd

import scipy
from scipy import sparse

import numpy as np
from sklearn.model_selection import TimeSeriesSplit

import pickle

In [2]:
# get article from yesterday

#https://understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-23-2023

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

yesterday_day = yesterday.day
yesterday_month = yesterday.month
yesterday_year = yesterday.year


In [3]:
file = text_processing.get_article_from_yesterday(yesterday_day,yesterday_month,yesterday_year)

In [4]:
data = text_processing.read_html(file)

In [5]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

In [6]:
def preprocess_all_text(data):
    pattern = "\[(\d+)\]"


    data['main_html_v1'] = data['main_html'].apply(lambda x: re.sub(pattern,"",str(x)))
    data['main_html_v2'] = data['main_html_v1'].apply(lambda x: re.sub(r'http(\S+.*\s)',"",x))
    data['main_html_v3'] = data['main_html_v2'].apply(lambda x: re.sub(r'2022|2023|©2022|©2023|\xa0|\n',"",x))

    data['main_html_v4'] = data['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
    
    data['main_html_v5'] = data['main_html_v4'].apply(lambda x: text_processing.remove_names_and_dates(x))
    
    return data

In [7]:
data_preprocessed = preprocess_all_text(data)

In [8]:
#data_preprocessed.head()

In [9]:
data_preprocessed = data_preprocessed.drop(['main_html_v1','main_html_v2','main_html_v3','main_html_v4'],axis=1)

In [10]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikita_voitishyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
data_preprocessed['report_text_lemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"lemm"))
data_preprocessed['report_text_stemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"stemm"))

In [12]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...


In [13]:
docs = data_preprocessed['report_text_lemm'].tolist()

In [14]:
#tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
#cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

In [15]:
# CountVectorizer + tfidf:create a vecror with numbers

cv = CountVectorizer()
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

tf_idf_vector = tfidf_transformer.transform(word_count_vector)

#<340x7401 sparse matrix of type '<class 'numpy.float64'>'
#	with 213470 stored elements in Compressed Sparse Row format>

In [16]:
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [17]:
feature_names = cv.get_feature_names()
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [18]:
data_preprocessed['keywords'] = data_preprocessed['report_text_stemm'].apply(lambda x: text_processing.convert_doc_to_vector(x,feature_names,tf_idf_vector))

In [19]:
data_preprocessed['keywords'][0]

{'russian': 0.708,
 'forc': 0.275,
 'ukrainian': 0.245,
 'twenty': 0.168,
 'april': 0.153,
 'claim': 0.132,
 'militari': 0.127,
 'five': 0.122,
 'report': 0.107,
 'two': 0.092}

In [20]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':..."


#### Part of scropt: 3_final_preprocess

In [21]:
data_preprocessed["date_datetime"] = pd.to_datetime(data_preprocessed["date"])
data_preprocessed['date_tomorrow_datetime'] = data_preprocessed['date_datetime'].apply(lambda x: x+datetime.timedelta(days=1))
data_preprocessed = data_preprocessed.rename(columns = {"date_datetime":"report_date"})

In [22]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords,report_date,date_tomorrow_datetime
0,2023-04-25,russian_offensive_campaign_assessment_April_25...,"Russian Offensive Campaign Assessment, April 2...","Russian Offensive Campaign Assessment, April 2...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, April ...",russian offens campaign ass april twenty five...,russian offen campaign assess april twenti fi...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':...",2023-04-25,2023-04-26


In [23]:
#data_preprocesdf_isw_short = data_preprocessed[['date','date_tomorrow_datetime','keywords','report_text_lemm']]

In [24]:
data_vectorised = tf_idf_vector.toarray()
vectors_df = pd.DataFrame(data_vectorised)
vectors_df['date'] = pd.to_datetime(today)


In [25]:
vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,831,832,833,834,835,836,837,838,839,date
0,0.005094,0.005094,0.015281,0.005094,0.005094,0.005094,0.010188,0.005094,0.010188,0.005094,...,0.005094,0.030563,0.005094,0.015281,0.015281,0.005094,0.010188,0.005094,0.025469,2023-04-26


In [26]:
df_isw_short = data_preprocessed[['date','report_text_lemm','keywords','date_tomorrow_datetime']]

In [27]:
df_isw_short.head()

,date,report_text_lemm,keywords,date_tomorrow_datetime
0,2023-04-25,russian offens campaign ass april twenty five...,"{'russian': 0.708, 'forc': 0.275, 'ukrainian':...",2023-04-26


In [28]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

#Uncommit if you want to load exist matrix
tfidf_vector = scipy.sparse.load_npz('data/matrix/tfidf_vector_train.npz')

In [29]:
import pytz

In [30]:
model = pickle.load(open("models/4_rf_3.1f.pkl","rb"))

/Users/nikita_voitishyn/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/nikita_voitishyn/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [31]:
tf_idf_vector

<1x840 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [37]:
cities = ['Крим','Вінниччина','Волинь','Дніпропетровщина','Донеччина','Житомирщина','Закарпаття', 'Запоріжжя',
          'Івано-Франківщина','Київщина','Кіровоградщина','Луганщина','Львівщина','Миколаївщина','Одещина','Полтавщина',
          'Рівненщина','Сумщина','Тернопільщина','Харківщина','Херсонщина','Хмельниччина','Черкащина',
          'Буковина','Чернігівщина']

# date = datetime.date.today()
date = today = datetime.date.today()
result = {}
for city in cities:

    
    df_weather_final = get_weather.get_weather_for_12_hours(city,date)
    
    

    df_weather_final['key']=1
    df_isw_short['key']=1
    
    df_all = df_weather_final.merge(df_isw_short, how = 'left', left_on = 'key', right_on = 'key')

    to_drop=['key','date','date_tomorrow_datetime','keywords','report_text_lemm']

    if 'sunrise' in df_all.columns:
        exceptions = ['sunset','sunrise']
        to_drop.extend(exceptions)

    df_weather_matrix_v1 = df_all.drop(to_drop, axis = 1)
    df_weather_matrix_v1= df_weather_matrix_v1[['day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity',
           'day_precip', 'day_precipcover', 'day_solarradiation',
           'day_solarenergy', 'day_uvindex', 'hour_temp', 'hour_humidity',
           'hour_dew', 'hour_precip', 'hour_precipprob', 'hour_snow',
           'hour_snowdepth', 'hour_windgust', 'hour_windspeed', 'hour_winddir',
           'hour_pressure', 'hour_visibility', 'hour_cloudcover',
           'hour_solarradiation', 'hour_uvindex', 'hour_severerisk','region_id','hour_datetimeEpoch']]
    #df_weather_matrix_v1.fillna(0, inplace=True)
    
    cv_vector_model = cv.transform(df_all['report_text_lemm'].values.astype('U'))
    #TF_IDF_MODEL = tfidf.transform(cv_vector_model)

    df_weather_matrix_v1_csr = scipy.sparse.csr_matrix(df_weather_matrix_v1)
    df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_v1_csr, tfidf_vector), format='csr')
    
    
    #predict
    predicted = model.predict(df_all_data_csr)
    
    current_time = pd.Timestamp.now()

    hours = []
    for i in range(12):
        hour = current_time + datetime.timedelta(hours=i)
        hours.append(hour) 
    
        
    result[city] = dict(zip(hours, predicted))
    
    
    

Weather data for the 
region Крим; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready


/var/folders/5y/jfstwx3n6x5bg_8mdp2wq81w0000gn/T/ipykernel_99197/3939247321.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isw_short['key']=1


Weather data for the 
region Вінниччина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Волинь; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Дніпропетровщина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Донеччина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Житомирщина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Закарпаття; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Запоріжжя; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Івано-Франківщина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Київщина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the 
region Кіровоградщина; 
start_date 2023-04-26; 
end_date 2023-04-26; 
is ready
Weather data for the

In [38]:
result

{'Крим': {Timestamp('2023-04-26 22:25:31.526357'): 0,
  Timestamp('2023-04-26 23:25:31.526357'): 0,
  Timestamp('2023-04-27 00:25:31.526357'): 0,
  Timestamp('2023-04-27 01:25:31.526357'): 0,
  Timestamp('2023-04-27 02:25:31.526357'): 0,
  Timestamp('2023-04-27 03:25:31.526357'): 0,
  Timestamp('2023-04-27 04:25:31.526357'): 0,
  Timestamp('2023-04-27 05:25:31.526357'): 0,
  Timestamp('2023-04-27 06:25:31.526357'): 0,
  Timestamp('2023-04-27 07:25:31.526357'): 0,
  Timestamp('2023-04-27 08:25:31.526357'): 0,
  Timestamp('2023-04-27 09:25:31.526357'): 0},
 'Вінниччина': {Timestamp('2023-04-26 22:25:53.597775'): 0,
  Timestamp('2023-04-26 23:25:53.597775'): 0,
  Timestamp('2023-04-27 00:25:53.597775'): 0,
  Timestamp('2023-04-27 01:25:53.597775'): 0,
  Timestamp('2023-04-27 02:25:53.597775'): 0,
  Timestamp('2023-04-27 03:25:53.597775'): 0,
  Timestamp('2023-04-27 04:25:53.597775'): 0,
  Timestamp('2023-04-27 05:25:53.597775'): 0,
  Timestamp('2023-04-27 06:25:53.597775'): 0,
  Timestamp

In [39]:
result = pd.DataFrame(result)
result.to_csv('results.txt', sep='\t', index=False)